In [36]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
from pandas.api.types import CategoricalDtype
from pyecharts import options as opts
from pyecharts.charts import Bar

import itertools

# 读取Excel文件
def read_excel_file(path,sheet):    
    ex = pd.ExcelFile(path)
    df = ex.parse(sheet)
    return df

# 获取分裂时间
def split_date_to_year_month_day(df, date_time_columns):
    for item in date_time_columns:            
        df[item] = pd.to_datetime(df[item])
        df[item+'年'] = df[item].dt.year
        df[item+'季'] = df[item].dt.quarter
        df[item+'月'] = df[item].dt.month
        df[item+'周'] = df[item].dt.isocalendar().week
        df[item + '日'] = df[item].dt.day
    return df

# 获取组织分裂
def split_csg_organization(df_object,organization_columns):
    for item in organization_columns:        
        df_object=df_object.join(df_object.loc[:,item].str.strip('/').str.split('/',expand=True).rename({0:item+'公司',1:item+'供电局',2:item+'分',3:item+'科室',4:item+'班组'},axis=1))
    return df_object

# 按要求排序
def sort_list(series_or_dataframe, list_sort, column_sort=None):
    """
    作者：孔令
    版本:v0.6
    功能：按照指定列和排序列表对Series或DataFrame排序;
    参数：series_or_dataframe Type Series or DataFrame,需要排序的数据;
         list_sort 排序列表集合，[[]]形式；
         column 需要排序的列
    返回：series，Type Series或dataframe,Type DataFrame
    """
    # 判断series_or_dataframe是否为pd.Series
    if isinstance(series_or_dataframe, pd.Series):
        # 将pd.Series转换成pd.DataFrame
        sort_list_df = pd.DataFrame(series_or_dataframe)
        # 重置索引，并保留索引
        sort_list_df = sort_list_df.reset_index()
        # 将列名改为index,values
        sort_list_df.columns = ['index', 'values']
        # 选出排序列中数据在排序列表中的数据，为了后续不在其中的不变为NAN
        sort_list_df1 = sort_list_df[sort_list_df['index'].isin(list_sort)].copy()
        # 选出排序列表中数据不在排序列表中的数据
        sort_list_df2 = sort_list_df[~sort_list_df['index'].isin(list_sort)].copy()
        # 将index排序列转换为category数据类型
        sort_list_df1['index'] = sort_list_df1['index'].astype('category')
        # 将index列按照list_sort排序
        sort_list_df1['index'].cat.set_categories(list_sort, inplace=True)
        sort_list_df1.sort_values('index', ascending=True, inplace=True)
        # 合并两个数据
        sort_list_df = pd.concat([sort_list_df1, sort_list_df2])
        # 转回Series类型
        series = pd.Series(sort_list_df['values'].values, index=sort_list_df['index'])
        return series
    # 判断series_or_dataframe是否为pd.DataFrame
    elif isinstance(series_or_dataframe, pd.DataFrame):
        # 将排序列中没有在排序列表的数据加入排序列表
        for i in range(len(column_sort)):
            list_sort[i] = list_sort[i] + list(
                set(series_or_dataframe[column_sort[i]]).difference(set(list_sort[i])))

        for i in range(len(column_sort)):
            # 将排序列表转换为CategoricalDtype类型
            cat_order = CategoricalDtype(
                list_sort[i],
                ordered=True
            )
            # 将排序列转换为排序列表类型
            series_or_dataframe[column_sort[i]] = series_or_dataframe[column_sort[i]].astype(cat_order)
        # 按照排序列表排序
        dataframe = series_or_dataframe.sort_values(column_sort, axis=0, ascending=[True] * (len(column_sort)))
        return dataframe

# 补齐统计缺失值
def fill_count_null_zero(df,iter_columns,count_columns,null_list):
    """
    df:需要统计的DataFrame
    iter_columns：需要笛卡尔积的列
    count_columns：需要补充缺失的列及值列
    null_list：缺失值内的具体值
    """
    set_list = []
    # 形成笛卡尔list
    for column in iter_columns:        
        set_list.append(set(df[column]))    
    # 用itertools.product形成笛卡尔积
    for i in itertools.product(*set_list):
        # 补充缺失值
        for item in null_list:            
            new = pd.DataFrame(
                    [
                        list(i)+[item]+[0]
                    
                    ],columns=iter_columns+count_columns)
            # 增加补充缺失值
            df=df.append(new,ignore_index=True)
    # 重新进行组合
    df = df.groupby(iter_columns+[count_columns[0]])[count_columns[1]].sum().reset_index()
    return df

def fill_count_full_zero(df,iter_columns,count_columns,full_list):
    """
    df:需要统计的DataFrame
    iter_columns：需要笛卡尔积的列
    count_columns：值列
    null_list：所有列的具体值
    """
    for item in itertools.product(*full_list):
        new = pd.DataFrame(
                    [
                        list(item)+[0]
                    
                    ],columns=iter_columns+count_columns)
        df=df.append(new,ignore_index=True)
    # 重新进行组合
    df = df.groupby(iter_columns)[count_columns].sum().reset_index()
    return df

# echarts bar图
def echarts_bar(s):   
    x = s.index.tolist()
    y = s.values.tolist()
    print(x,y)
    c = (
        Bar()
        .add_xaxis(['A',])
        .add_yaxis("违章分类", y)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Bar-DataZoom（slider+inside）"),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
        )
       
    )
    c.render_notebook()


# 导入排序及替换库
path_sort= r'D:\JGY\600-Data\002-in输入文件\02-work工作\03-document工作文档\000-文件地图\昆明供电局排序表.xlsx'
sheet_sort = '安全监督系统'
df_sort = read_excel_file(path_sort,sheet_sort)
df_dict = df_sort.set_index('安监系统单位名称')['简称'].to_dict()
list_sort = df_sort['安监系统单位名称'].tolist()

# 导入监督计划
path_plan= r'D:\JGY\600-Data\002-in输入文件\02-work工作\01-system工作系统数据\001-安全监督计划\昆明供电局安全监督计划.xlsx'
sheet_plan = 'Sheet1'
df_plan = read_excel_file(path_plan,sheet_plan)
# 分裂组织类型列
df_plan = split_csg_organization(df_plan,['检查部门/单位'])
# 分裂时间性质列
df_plan = split_date_to_year_month_day(df_plan,['计划开始时间','计划结束时间'])
df_plan_count = df_plan.groupby(['计划开始时间年','计划开始时间月'])['计划开始时间'].count().reset_index(name='检查次数')
df_plan_count.columns = ['年','月','检查次数']
df_plan_count


# 导入违章库
path_illegal= r'D:\JGY\600-Data\002-in输入文件\02-work工作\01-system工作系统数据\002-违章台账\违章台账.xlsx'
sheet_illegal = 'Sheet1'
df_illegal = read_excel_file(path_illegal,sheet_illegal)
# 分裂组织类型列
df_illegal = split_csg_organization(df_illegal,['督查单位','项目管理单位','违章单位（班组）'])
# 分裂时间性质列
df_illegal = split_date_to_year_month_day(df_illegal,['违章时间'])
# 转换数据类型
df_illegal = df_illegal.convert_dtypes()
# 简化组织名称
df_illegal['项目管理单位分'] = df_illegal['项目管理单位分'].map(df_dict)

total = len(df_illegal[df_illegal['违章时间年']== 2021])
text="2021年违章总数："
text
total
df_time = df_illegal.groupby(['违章时间年','违章时间月','督查层面'])['违章时间'].count().reset_index(name='次数')
df_time.columns = ['年','月','暴露层级','违章次数']
year_list = [2020,2021]
month_list = [1,2,3,4,5,6,7,8,9,10,11,12]
exp_list = ['公司级','省级','地市级','县区级','班站所']
org_list = df_dict
df_time_level = fill_count_full_zero(df_time, ['年','月','暴露层级'],['违章次数'],[year_list,month_list,exp_list])
df_time_level = sort_list(df_time_level, [year_list,month_list,exp_list],column_sort=['年','月','暴露层级'] ).reset_index(drop=True)
df_time_level['状态'] = '正常'
#df_time_level.to_excel(r'D:\JGY\600-Data\006-temporary临时文件\1.xlsx',engine='openpyxl')
# for i in range(len(df_time_level)-1):
#     if (i%5)!=0:               
#         if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
#             if ((i+1)%5)!=0:
#                 df_time_level.loc[i+1,'状态'] = '异常' 
for i in range(len(df_time_level)-5):
    if (i%5)==1:               
        if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+1,'状态'] = '异常'
        elif df_time_level.iloc[i+2,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+2,'状态'] = '异常' 
        elif df_time_level.iloc[i+3,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+3,'状态'] = '异常' 
   
    elif (i%5)==2:
        if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+1,'状态'] = '异常'
        elif df_time_level.iloc[i+2,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+2,'状态'] = '异常' 

    elif (i%5)==3:
        if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+1,'状态'] = '异常'
    
df_time_level_i = df_time_level[df_time_level['状态'] == '异常']
df_time_level_i
#df_time_level.to_excel(r'D:\JGY\600-Data\006-temporary临时文件\2.xlsx',engine='openpyxl')
#df_time_level.groupby(['年','月'])
 

,年,月,检查次数
0,2021,1,6801
1,2021,2,5020


'2021年违章总数：'

419

# 违章不满足金字塔的单位

In [42]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
from pandas.api.types import CategoricalDtype
from pyecharts import options as opts
from pyecharts.charts import Bar

import itertools

# 读取Excel文件
def read_excel_file(path,sheet):    
    ex = pd.ExcelFile(path)
    df = ex.parse(sheet)
    return df

# 获取分裂时间
def split_date_to_year_month_day(df, date_time_columns):
    for item in date_time_columns:            
        df[item] = pd.to_datetime(df[item])
        df[item+'年'] = df[item].dt.year
        df[item+'季'] = df[item].dt.quarter
        df[item+'月'] = df[item].dt.month
        df[item+'周'] = df[item].dt.isocalendar().week
        df[item + '日'] = df[item].dt.day
    return df

# 获取组织分裂
def split_csg_organization(df_object,organization_columns):
    for item in organization_columns:        
        df_object=df_object.join(df_object.loc[:,item].str.strip('/').str.split('/',expand=True).rename({0:item+'公司',1:item+'供电局',2:item+'分',3:item+'科室',4:item+'班组'},axis=1))
    return df_object

# 按要求排序
def sort_list(series_or_dataframe, list_sort, column_sort=None):
    """
    作者：孔令
    版本:v0.6
    功能：按照指定列和排序列表对Series或DataFrame排序;
    参数：series_or_dataframe Type Series or DataFrame,需要排序的数据;
         list_sort 排序列表集合，[[]]形式；
         column 需要排序的列
    返回：series，Type Series或dataframe,Type DataFrame
    """
    # 判断series_or_dataframe是否为pd.Series
    if isinstance(series_or_dataframe, pd.Series):
        # 将pd.Series转换成pd.DataFrame
        sort_list_df = pd.DataFrame(series_or_dataframe)
        # 重置索引，并保留索引
        sort_list_df = sort_list_df.reset_index()
        # 将列名改为index,values
        sort_list_df.columns = ['index', 'values']
        # 选出排序列中数据在排序列表中的数据，为了后续不在其中的不变为NAN
        sort_list_df1 = sort_list_df[sort_list_df['index'].isin(list_sort)].copy()
        # 选出排序列表中数据不在排序列表中的数据
        sort_list_df2 = sort_list_df[~sort_list_df['index'].isin(list_sort)].copy()
        # 将index排序列转换为category数据类型
        sort_list_df1['index'] = sort_list_df1['index'].astype('category')
        # 将index列按照list_sort排序
        sort_list_df1['index'].cat.set_categories(list_sort, inplace=True)
        sort_list_df1.sort_values('index', ascending=True, inplace=True)
        # 合并两个数据
        sort_list_df = pd.concat([sort_list_df1, sort_list_df2])
        # 转回Series类型
        series = pd.Series(sort_list_df['values'].values, index=sort_list_df['index'])
        return series
    # 判断series_or_dataframe是否为pd.DataFrame
    elif isinstance(series_or_dataframe, pd.DataFrame):
        # 将排序列中没有在排序列表的数据加入排序列表
        for i in range(len(column_sort)):
            list_sort[i] = list_sort[i] + list(
                set(series_or_dataframe[column_sort[i]]).difference(set(list_sort[i])))

        for i in range(len(column_sort)):
            # 将排序列表转换为CategoricalDtype类型
            cat_order = CategoricalDtype(
                list_sort[i],
                ordered=True
            )
            # 将排序列转换为排序列表类型
            series_or_dataframe[column_sort[i]] = series_or_dataframe[column_sort[i]].astype(cat_order)
        # 按照排序列表排序
        dataframe = series_or_dataframe.sort_values(column_sort, axis=0, ascending=[True] * (len(column_sort)))
        return dataframe

# 补齐统计缺失值
def fill_count_null_zero(df,iter_columns,count_columns,null_list):
    """
    df:需要统计的DataFrame
    iter_columns：需要笛卡尔积的列
    count_columns：需要补充缺失的列及值列
    null_list：缺失值内的具体值
    """
    set_list = []
    # 形成笛卡尔list
    for column in iter_columns:        
        set_list.append(set(df[column]))    
    # 用itertools.product形成笛卡尔积
    for i in itertools.product(*set_list):
        # 补充缺失值
        for item in null_list:            
            new = pd.DataFrame(
                    [
                        list(i)+[item]+[0]
                    
                    ],columns=iter_columns+count_columns)
            # 增加补充缺失值
            df=df.append(new,ignore_index=True)
    # 重新进行组合
    df = df.groupby(iter_columns+[count_columns[0]])[count_columns[1]].sum().reset_index()
    return df

def fill_count_full_zero(df,iter_columns,count_columns,full_list):
    """
    df:需要统计的DataFrame
    iter_columns：需要笛卡尔积的列
    count_columns：值列
    null_list：所有列的具体值
    """
    for item in itertools.product(*full_list):
        new = pd.DataFrame(
                    [
                        list(item)+[0]
                    
                    ],columns=iter_columns+count_columns)
        df=df.append(new,ignore_index=True)
    # 重新进行组合
    df = df.groupby(iter_columns)[count_columns].sum().reset_index()
    return df

# echarts bar图
def echarts_bar(s):   
    x = s.index.tolist()
    y = s.values.tolist()
    print(x,y)
    c = (
        Bar()
        .add_xaxis(['A',])
        .add_yaxis("违章分类", y)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Bar-DataZoom（slider+inside）"),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
        )
       
    )
    c.render_notebook()


# 导入排序及替换库
path_sort= r'D:\JGY\600-Data\002-in输入文件\02-work工作\03-document工作文档\000-文件地图\昆明供电局排序表.xlsx'
sheet_sort = '安全监督系统'
df_sort = read_excel_file(path_sort,sheet_sort)
df_dict = df_sort.set_index('安监系统单位名称')['简称'].to_dict()
list_sort = df_sort['安监系统单位名称'].tolist()
list_sort_simple = df_sort['简称'].tolist()

# 导入监督计划
path_plan= r'D:\JGY\600-Data\002-in输入文件\02-work工作\01-system工作系统数据\001-安全监督计划\昆明供电局安全监督计划.xlsx'
sheet_plan = 'Sheet1'
df_plan = read_excel_file(path_plan,sheet_plan)
# 分裂组织类型列
df_plan = split_csg_organization(df_plan,['检查部门/单位'])
# 分裂时间性质列
df_plan = split_date_to_year_month_day(df_plan,['计划开始时间','计划结束时间'])
df_plan_count = df_plan.groupby(['计划开始时间年','计划开始时间月'])['计划开始时间'].count().reset_index(name='检查次数')
df_plan_count.columns = ['年','月','检查次数']
df_plan_count


# 导入违章库
path_illegal= r'D:\JGY\600-Data\002-in输入文件\02-work工作\01-system工作系统数据\002-违章台账\违章台账.xlsx'
sheet_illegal = 'Sheet1'
df_illegal = read_excel_file(path_illegal,sheet_illegal)
# 分裂组织类型列
df_illegal = split_csg_organization(df_illegal,['督查单位','项目管理单位','违章单位（班组）'])
# 分裂时间性质列
df_illegal = split_date_to_year_month_day(df_illegal,['违章时间'])
# 转换数据类型
df_illegal = df_illegal.convert_dtypes()
# 简化组织名称
df_illegal['项目管理单位分'] = df_illegal['项目管理单位分'].map(df_dict)

total = len(df_illegal[df_illegal['违章时间年']== 2021])
text="2021年违章总数："
text
total
df_org_time = df_illegal.groupby(['项目管理单位分','违章时间年','违章时间月','督查层面'])['违章时间'].count().reset_index(name='次数')
df_org_time.columns = ['单位','年','月','暴露层级','违章次数']
year_list = [2020,2021]
month_list = [1,2,3,4,5,6,7,8,9,10,11,12]
exp_list = ['公司级','省级','地市级','县区级','班站所']
org_list = list_sort_simple
df_org_time_level = fill_count_full_zero(df_org_time, ['单位','年','月','暴露层级'],['违章次数'],[org_list,year_list,month_list,exp_list])
df_org_time_level = sort_list(df_org_time_level, [org_list,year_list,month_list,exp_list],column_sort=['单位','年','月','暴露层级'] ).reset_index(drop=True)
df_org_time_level['状态'] = '正常'
df_org_time_level['原因'] = ''
#df_time_level.to_excel(r'D:\JGY\600-Data\006-temporary临时文件\1.xlsx',engine='openpyxl')
# for i in range(len(df_time_level)-1):
#     if (i%5)!=0:               
#         if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
#             if ((i+1)%5)!=0:
#                 df_time_level.loc[i+1,'状态'] = '异常' 
for i in range(len(df_org_time_level)-5):
    if (i%5)==1:               
        if df_org_time_level.iloc[i+1,4]<=df_org_time_level.iloc[i,4] and df_org_time_level.iloc[i,4]!=0:
            df_org_time_level.loc[i+1,'状态'] = '异常'
            df_org_time_level.loc[i+1,'原因'] = f"{df_org_time_level.iloc[i,3]}发现{df_org_time_level.iloc[i,4]}次，大于等于{df_org_time_level.iloc[i+1,3]}发现{df_org_time_level.iloc[i+1,4]}次"
        elif df_org_time_level.iloc[i+2,4]<=df_org_time_level.iloc[i,4] and df_org_time_level.iloc[i,4]!=0:
            df_org_time_level.loc[i+2,'状态'] = '异常' 
            df_org_time_level.loc[i+2,'原因'] = f"{df_org_time_level.iloc[i,3]}发现{df_org_time_level.iloc[i,4]}次，大于等于{df_org_time_level.iloc[i+2,3]}发现{df_org_time_level.iloc[i+2,4]}次"
        elif df_org_time_level.iloc[i+3,4]<=df_org_time_level.iloc[i,4] and df_org_time_level.iloc[i,4]!=0:
            df_org_time_level.loc[i+3,'状态'] = '异常' 
            df_org_time_level.loc[i+3,'原因'] = f"{df_org_time_level.iloc[i,3]}发现{df_org_time_level.iloc[i,4]}次，大于等于{df_org_time_level.iloc[i+3,3]}发现{df_org_time_level.iloc[i+3,4]}次"
   
    elif (i%5)==2:
        if df_org_time_level.iloc[i+1,4]<=df_org_time_level.iloc[i,4] and df_org_time_level.iloc[i,4]!=0:
            df_org_time_level.loc[i+1,'状态'] = '异常'
            df_org_time_level.loc[i+1,'原因'] = f"{df_org_time_level.iloc[i,3]}发现{df_org_time_level.iloc[i,4]}次，大于等于{df_org_time_level.iloc[i+1,3]}发现{df_org_time_level.iloc[i+1,4]}次"
        elif df_org_time_level.iloc[i+2,4]<=df_org_time_level.iloc[i,4] and df_org_time_level.iloc[i,4]!=0:
            df_org_time_level.loc[i+2,'状态'] = '异常' 
            df_org_time_level.loc[i+2,'原因'] = f"{df_org_time_level.iloc[i,3]}发现{df_org_time_level.iloc[i,4]}次，大于等于{df_org_time_level.iloc[i+2,3]}发现{df_org_time_level.iloc[i+2,4]}次"

    elif (i%5)==3:
        if df_org_time_level.iloc[i+1,4]<=df_org_time_level.iloc[i,4] and df_org_time_level.iloc[i,4]!=0:
            df_org_time_level.loc[i+1,'状态'] = '异常'
            df_org_time_level.loc[i+1,'原因'] = f"{df_org_time_level.iloc[i,3]}发现{df_org_time_level.iloc[i,4]}次，大于等于{df_org_time_level.iloc[i+1,3]}发现{df_org_time_level.iloc[i+1,4]}次"
df_org_time_level.to_excel(r'D:\JGY\600-Data\006-temporary临时文件\2.xlsx',engine='openpyxl')    
df_org_time_level_i = df_org_time_level[df_org_time_level['状态'] == '异常']
df_org_time_level_i
#df_time_level.to_excel(r'D:\JGY\600-Data\006-temporary临时文件\2.xlsx',engine='openpyxl')
#df_time_level.groupby(['年','月'])
 

,年,月,检查次数
0,2021,1,6801
1,2021,2,5020


'2021年违章总数：'

419

,单位,年,月,暴露层级,违章次数,状态,原因
1854,输电所,2020,11,班站所,3,异常,县区级发现9次，大于等于班站所发现3次
1859,输电所,2020,12,班站所,0,异常,县区级发现4次，大于等于班站所发现0次
1864,输电所,2021,1,班站所,1,异常,县区级发现2次，大于等于班站所发现1次
1992,变电一所,2021,3,地市级,0,异常,省级发现1次，大于等于地市级发现0次
2103,变电二所,2021,1,县区级,0,异常,地市级发现1次，大于等于县区级发现0次
...,...,...,...,...,...,...,...
5088,寻甸供电局,2020,10,县区级,5,异常,地市级发现5次，大于等于县区级发现5次
5089,寻甸供电局,2020,10,班站所,5,异常,县区级发现5次，大于等于班站所发现5次
5094,寻甸供电局,2020,11,班站所,3,异常,县区级发现8次，大于等于班站所发现3次
5099,寻甸供电局,2020,12,班站所,1,异常,县区级发现3次，大于等于班站所发现1次


# 违章前三的原因

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
from pandas.api.types import CategoricalDtype
from pyecharts import options as opts
from pyecharts.charts import Bar

import itertools

# 读取Excel文件
def read_excel_file(path,sheet):    
    ex = pd.ExcelFile(path)
    df = ex.parse(sheet)
    return df

# 获取分裂时间
def split_date_to_year_month_day(df, date_time_columns):
    for item in date_time_columns:            
        df[item] = pd.to_datetime(df[item])
        df[item+'年'] = df[item].dt.year
        df[item+'季'] = df[item].dt.quarter
        df[item+'月'] = df[item].dt.month
        df[item+'周'] = df[item].dt.isocalendar().week
        df[item + '日'] = df[item].dt.day
    return df

# 获取组织分裂
def split_csg_organization(df_object,organization_columns):
    for item in organization_columns:        
        df_object=df_object.join(df_object.loc[:,item].str.strip('/').str.split('/',expand=True).rename({0:item+'公司',1:item+'供电局',2:item+'分',3:item+'科室',4:item+'班组'},axis=1))
    return df_object

# 按要求排序
def sort_list(series_or_dataframe, list_sort, column_sort=None):
    """
    作者：孔令
    版本:v0.6
    功能：按照指定列和排序列表对Series或DataFrame排序;
    参数：series_or_dataframe Type Series or DataFrame,需要排序的数据;
         list_sort 排序列表集合，[[]]形式；
         column 需要排序的列
    返回：series，Type Series或dataframe,Type DataFrame
    """
    # 判断series_or_dataframe是否为pd.Series
    if isinstance(series_or_dataframe, pd.Series):
        # 将pd.Series转换成pd.DataFrame
        sort_list_df = pd.DataFrame(series_or_dataframe)
        # 重置索引，并保留索引
        sort_list_df = sort_list_df.reset_index()
        # 将列名改为index,values
        sort_list_df.columns = ['index', 'values']
        # 选出排序列中数据在排序列表中的数据，为了后续不在其中的不变为NAN
        sort_list_df1 = sort_list_df[sort_list_df['index'].isin(list_sort)].copy()
        # 选出排序列表中数据不在排序列表中的数据
        sort_list_df2 = sort_list_df[~sort_list_df['index'].isin(list_sort)].copy()
        # 将index排序列转换为category数据类型
        sort_list_df1['index'] = sort_list_df1['index'].astype('category')
        # 将index列按照list_sort排序
        sort_list_df1['index'].cat.set_categories(list_sort, inplace=True)
        sort_list_df1.sort_values('index', ascending=True, inplace=True)
        # 合并两个数据
        sort_list_df = pd.concat([sort_list_df1, sort_list_df2])
        # 转回Series类型
        series = pd.Series(sort_list_df['values'].values, index=sort_list_df['index'])
        return series
    # 判断series_or_dataframe是否为pd.DataFrame
    elif isinstance(series_or_dataframe, pd.DataFrame):
        # 将排序列中没有在排序列表的数据加入排序列表
        for i in range(len(column_sort)):
            list_sort[i] = list_sort[i] + list(
                set(series_or_dataframe[column_sort[i]]).difference(set(list_sort[i])))

        for i in range(len(column_sort)):
            # 将排序列表转换为CategoricalDtype类型
            cat_order = CategoricalDtype(
                list_sort[i],
                ordered=True
            )
            # 将排序列转换为排序列表类型
            series_or_dataframe[column_sort[i]] = series_or_dataframe[column_sort[i]].astype(cat_order)
        # 按照排序列表排序
        dataframe = series_or_dataframe.sort_values(column_sort, axis=0, ascending=[True] * (len(column_sort)))
        return dataframe

# 补齐统计缺失值
def fill_count_null_zero(df,iter_columns,count_columns,null_list):
    """
    df:需要统计的DataFrame
    iter_columns：需要笛卡尔积的列
    count_columns：需要补充缺失的列及值列
    null_list：缺失值内的具体值
    """
    set_list = []
    # 形成笛卡尔list
    for column in iter_columns:        
        set_list.append(set(df[column]))    
    # 用itertools.product形成笛卡尔积
    for i in itertools.product(*set_list):
        # 补充缺失值
        for item in null_list:            
            new = pd.DataFrame(
                    [
                        list(i)+[item]+[0]
                    
                    ],columns=iter_columns+count_columns)
            # 增加补充缺失值
            df=df.append(new,ignore_index=True)
    # 重新进行组合
    df = df.groupby(iter_columns+[count_columns[0]])[count_columns[1]].sum().reset_index()
    return df

def fill_count_full_zero(df,iter_columns,count_columns,full_list):
    """
    df:需要统计的DataFrame
    iter_columns：需要笛卡尔积的列
    count_columns：值列
    null_list：所有列的具体值
    """
    for item in itertools.product(*full_list):
        new = pd.DataFrame(
                    [
                        list(item)+[0]
                    
                    ],columns=iter_columns+count_columns)
        df=df.append(new,ignore_index=True)
    # 重新进行组合
    df = df.groupby(iter_columns)[count_columns].sum().reset_index()
    return df

# echarts bar图
def echarts_bar(s):   
    x = s.index.tolist()
    y = s.values.tolist()
    print(x,y)
    c = (
        Bar()
        .add_xaxis(['A',])
        .add_yaxis("违章分类", y)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Bar-DataZoom（slider+inside）"),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
        )
       
    )
    c.render_notebook()


# 导入排序及替换库
path_sort= r'D:\JGY\600-Data\002-in输入文件\02-work工作\03-document工作文档\000-文件地图\昆明供电局排序表.xlsx'
sheet_sort = '安全监督系统'
df_sort = read_excel_file(path_sort,sheet_sort)
df_dict = df_sort.set_index('安监系统单位名称')['简称'].to_dict()
list_sort = df_sort['安监系统单位名称'].tolist()

# 导入监督计划
path_plan= r'D:\JGY\600-Data\002-in输入文件\02-work工作\01-system工作系统数据\001-安全监督计划\昆明供电局安全监督计划.xlsx'
sheet_plan = 'Sheet1'
df_plan = read_excel_file(path_plan,sheet_plan)
# 分裂组织类型列
df_plan = split_csg_organization(df_plan,['检查部门/单位'])
# 分裂时间性质列
df_plan = split_date_to_year_month_day(df_plan,['计划开始时间','计划结束时间'])
df_plan_count = df_plan.groupby(['计划开始时间年','计划开始时间月'])['计划开始时间'].count().reset_index(name='检查次数')
df_plan_count.columns = ['年','月','检查次数']
df_plan_count


# 导入违章库
path_illegal= r'D:\JGY\600-Data\002-in输入文件\02-work工作\01-system工作系统数据\002-违章台账\违章台账.xlsx'
sheet_illegal = 'Sheet1'
df_illegal = read_excel_file(path_illegal,sheet_illegal)
# 分裂组织类型列
df_illegal = split_csg_organization(df_illegal,['督查单位','项目管理单位','违章单位（班组）'])
# 分裂时间性质列
df_illegal = split_date_to_year_month_day(df_illegal,['违章时间'])
# 转换数据类型
df_illegal = df_illegal.convert_dtypes()
# 简化组织名称
df_illegal['项目管理单位分'] = df_illegal['项目管理单位分'].map(df_dict)

total = len(df_illegal[df_illegal['违章时间年']== 2021])
text="2021年违章总数："
text
total
df_time = df_illegal.groupby(['违章时间年','违章时间月','违章代码'])['违章时间'].count().reset_index(name='次数')
df_time.columns = ['年','月','违章代码','违章次数']
year_list = [2020,2021]
month_list = [1,2,3,4,5,6,7,8,9,10,11,12]
exp_list = ['公司级','省级','地市级','县区级','班站所']
org_list = df_dict
df_time_level = fill_count_full_zero(df_time, ['年','月','违章代码'],['违章次数'],[year_list,month_list,exp_list])
df_time_level = sort_list(df_time_level, [year_list,month_list,exp_list],column_sort=['年','月','暴露层级'] ).reset_index(drop=True)
df_time_level['状态'] = '正常'
#df_time_level.to_excel(r'D:\JGY\600-Data\006-temporary临时文件\1.xlsx',engine='openpyxl')
# for i in range(len(df_time_level)-1):
#     if (i%5)!=0:               
#         if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
#             if ((i+1)%5)!=0:
#                 df_time_level.loc[i+1,'状态'] = '异常' 
for i in range(len(df_time_level)-5):
    if (i%5)==1:               
        if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+1,'状态'] = '异常'
        elif df_time_level.iloc[i+2,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+2,'状态'] = '异常' 
        elif df_time_level.iloc[i+3,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+3,'状态'] = '异常' 
   
    elif (i%5)==2:
        if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+1,'状态'] = '异常'
        elif df_time_level.iloc[i+2,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+2,'状态'] = '异常' 

    elif (i%5)==3:
        if df_time_level.iloc[i+1,3]<=df_time_level.iloc[i,3] and df_time_level.iloc[i,3]!=0:
            df_time_level.loc[i+1,'状态'] = '异常'
    
df_time_level_i = df_time_level[df_time_level['状态'] == '异常']
df_time_level_i
#df_time_level.to_excel(r'D:\JGY\600-Data\006-temporary临时文件\2.xlsx',engine='openpyxl')
#df_time_level.groupby(['年','月'])
 